In [ ]:
import os
import random
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T

import timm
from torch_audiomentations import Compose, Gain, AddColoredNoise, PitchShift, Shift


sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 4)

class CFG:
    # General
    SEED = 42
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

    DATA_PATH = f"C:\\Users\\sinha\\Downloads\\tsda_dataset-20251015T052020Z-1-001\\tsda_dataset"
    AUDIO_PATH = os.path.join(DATA_PATH, "audio")
    META_PATH = os.path.join(DATA_PATH, "meta/sound_50.csv")
    

    SAMPLING_RATE = 44100
    N_FFT = 2048          
    WIN_LENGTH = 1024     
    HOP_LENGTH = 512
    N_MELS = 128
    
    
    NUM_CLASSES = 50
    BATCH_SIZE = 32
    EPOCHS = 50           
    LEARNING_RATE = 3e-4  
    WEIGHT_DECAY = 1e-2   

def set_seed(seed):
    """Sets the seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)   # <<< fixed
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CFG.SEED)
print(f"Using device: {CFG.DEVICE}")


mel_transform = T.MelSpectrogram(
    sample_rate=CFG.SAMPLING_RATE,
    n_fft=CFG.N_FFT,
    win_length=CFG.WIN_LENGTH,
    hop_length=CFG.HOP_LENGTH,
    n_mels=CFG.N_MELS
)
db_transform = T.AmplitudeToDB()

try:
    audio_files = os.listdir(CFG.AUDIO_PATH)
    print(f"Found {len(audio_files)} audio files in {CFG.AUDIO_PATH}")
    meta_files = os.listdir(os.path.join(CFG.DATA_PATH, "meta"))
    print(f"Found metadata files: {meta_files}")
except FileNotFoundError:
    print("Error: Dataset not found. Please check the DATA_PATH in the CFG class.")

# Load the metadata CSV file
df = pd.read_csv(CFG.META_PATH)

# Display the first few rows to understand its structure
print("Metadata DataFrame Head:")
print(df.head())


def get_model(num_classes=CFG.NUM_CLASSES, pretrained=True):
    """
    Loads a pre-trained EfficientNet-B0 model from the 'timm' library.
    The model's first convolutional layer is modified to accept 1-channel (grayscale)
    spectrograms instead of 3-channel RGB images.
    """
    model = timm.create_model(
        'efficientnet_b0',
        pretrained=pretrained,
        num_classes=num_classes,
        in_chans=1  # Key change: adapt for single-channel spectrograms
    )
    return model

# Instantiate the model to check its structure
model = get_model(num_classes=CFG.NUM_CLASSES)
print("\nUsing Pre-trained EfficientNet-B0 Model Architecture:")

class ESC50Dataset(Dataset):
    def __init__(self, df, data_path, fold_to_exclude, is_train=True, transform=None, waveform_augmentations=None):
        self.data_path = data_path
        if is_train:
            self.df = df[df['fold']!= fold_to_exclude].reset_index(drop=True)
        else:
            self.df = df[df['fold'] == fold_to_exclude].reset_index(drop=True)
            
        self.transform = transform
        self.waveform_augmentations = waveform_augmentations
        self.is_train = is_train
        self.target_length = 5 * CFG.SAMPLING_RATE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file_path = os.path.join(self.data_path, row['filename'])
        
        waveform, sr = torchaudio.load(file_path)  # shape: [channels, samples] or [samples]
        if waveform.dim() == 1:
            waveform = waveform.unsqueeze(0)  # -> [1, samples]
        
        if sr != CFG.SAMPLING_RATE:
            waveform = T.Resample(sr, CFG.SAMPLING_RATE)(waveform)
        
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)  # -> [1, samples]
        
        waveform = waveform.to(dtype=torch.float32)
        
        if self.is_train and self.waveform_augmentations is not None:
            added_batch = False
            if waveform.dim() == 2:  # [channels, samples]
                waveform = waveform.unsqueeze(0)  # -> [1, channels, samples]
                added_batch = True
    
            try:
                augmented = self.waveform_augmentations(waveform, sample_rate=CFG.SAMPLING_RATE)
            except TypeError:
                try:
                    augmented = self.waveform_augmentations(waveform)
                except TypeError:
                    augmented = self.waveform_augmentations(samples=waveform, sample_rate=CFG.SAMPLING_RATE)
            
            if isinstance(augmented, dict):
                if 'samples' in augmented:
                    augmented = augmented['samples']
                elif 'augmented_samples' in augmented:
                    augmented = augmented['augmented_samples']
                else:
                    raise RuntimeError(f"Unexpected augmenter return keys: {list(augmented.keys())}")
            
            if added_batch:
                augmented = augmented.squeeze(0)  # -> [channels, samples]
            
            waveform = augmented.to(dtype=torch.float32)
    
        n_samples = waveform.shape[1]
        if n_samples < self.target_length:
            pad_amount = self.target_length - n_samples
            waveform = torch.nn.functional.pad(waveform, (0, pad_amount))  
        else:
            waveform = waveform[:, : self.target_length]
        
        
        spectrogram = self.transform(waveform)   
        spectrogram = spectrogram.to(dtype=torch.float32)
        
        spectrogram = spectrogram.unsqueeze(0)           
        spectrogram = torch.nn.functional.interpolate(
            spectrogram, size=(224, 224), mode='bilinear', align_corners=False
        )                                                
        spectrogram = spectrogram.squeeze(0)             
        
        label = torch.tensor(row['target'], dtype=torch.long)
        return spectrogram, label


waveform_augmenter = Compose(
    transforms=[
        Gain(min_gain_in_db=-12.0, max_gain_in_db=12.0, p=0.5, output_type='tensor'),
        AddColoredNoise(min_snr_in_db=6.0, max_snr_in_db=20.0, p=0.45, output_type='tensor'),
        PitchShift(
            min_transpose_semitones=-2,
            max_transpose_semitones=2,
            p=0.35,
            sample_rate=CFG.SAMPLING_RATE,   # pass sample_rate here (required)
            output_type='tensor'
        ),
        Shift(min_shift=-0.2, max_shift=0.2, p=0.35, output_type='tensor'),
    ]
)

spec_transform = nn.Sequential(
    mel_transform,
    db_transform
)

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy

def validate_one_epoch(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy

# Main cross-validation loop
fold_accuracies = []

for fold in range(1, 6):
    print(f"\n===== FOLD {fold} =====")
    

    train_dataset = ESC50Dataset(df, CFG.AUDIO_PATH, fold_to_exclude=fold, is_train=True, 
                                 transform=spec_transform, waveform_augmentations=waveform_augmenter)
    val_dataset = ESC50Dataset(df, CFG.AUDIO_PATH, fold_to_exclude=fold, is_train=False, 
                               transform=spec_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    
    model = get_model(num_classes=CFG.NUM_CLASSES).to(CFG.DEVICE)
    
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    optimizer = optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE, weight_decay=CFG.WEIGHT_DECAY)
    
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.EPOCHS, eta_min=1e-6)
    
    best_val_accuracy = 0.0
    best_model_path = f"best_model_fold_{fold}.pth"

    for epoch in range(CFG.EPOCHS):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, CFG.DEVICE)
        
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{CFG.EPOCHS} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best validation accuracy: {best_val_accuracy:.4f}. Model saved.")
            
    fold_accuracies.append(best_val_accuracy)
    print(f"Best validation accuracy for fold {fold}: {best_val_accuracy:.4f}")

print("\n===================================")
print("Cross-Validation Complete.")
print(f"All fold accuracies: {fold_accuracies}")
print(f"Average CV Accuracy: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}")
print("===================================")


Using device: cuda
Found 2000 audio files in C:\Users\sinha\Downloads\tsda_dataset-20251015T052020Z-1-001\tsda_dataset\audio
Found metadata files: ['sound_50.csv', 'sound_human_annotations.xlsx']
Metadata DataFrame Head:
            filename  fold  target        category  esc10  src_file take
0   1-100032-A-0.wav     1       0             dog   True    100032    A
1  1-100038-A-14.wav     1      14  chirping_birds  False    100038    A
2  1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A
3  1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B
4  1-101296-A-19.wav     1      19    thunderstorm  False    101296    A

Using Pre-trained EfficientNet-B0 Model Architecture:

===== FOLD 1 =====


C:\Users\sinha\anaconda3\Lib\site-packages\torch_audiomentations\core\composition.py:42: FutureWarning: Transforms now expect an `output_type` argument that currently defaults to 'tensor', will default to 'dict' in v0.12, and will be removed in v0.13. Make sure to update your code to something like:
  >>> augment = Compose(..., output_type='dict')
  >>> augmented_samples = augment(samples).samples
  warnings.warn(


Training:   0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\sinha\anaconda3\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


Validation:   0%|          | 0/13 [00:00<?, ?it/s]

...

Epoch 19/50 | Train Loss: 0.8030, Train Acc: 0.9950 | Val Loss: 1.2265, Val Acc: 0.8400


Training:   0%|          | 0/50 [00:00<?, ?it/s]

Validation:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 20/50 | Train Loss: 0.7973, Train Acc: 0.9956 | Val Loss: 1.2233, Val Acc: 0.8500


Training:   0%|          | 0/50 [00:00<?, ?it/s]